In [29]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AksCompute

from azureml.core.workspace import Workspace

from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Model, Datastore, Dataset

subscription_id = 'YOUR_SUBSCRIPTION'
resource_group = 'YOUR_RESOURCE_GROUP'
workspace_name = 'YOUR_AML_WORKSPACE'

auth = InteractiveLoginAuthentication()


ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,auth= auth)
# ws = Workspace.get(name=workspace_name,
#                    subscription_id=subscription_id,
#                    resource_group=resource_group,auth= auth)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

datastore = ws.get_default_datastore()


Workspace name: YOUR_AML_WORKSPACE3
Azure region: eastus
Subscription id: YOUR_SUBSCRIPTION
Resource group: YOUR_RESOURCE_GROUP


In this cell, we create the environment using the dockerfile definition 

In [33]:
 #Ray Cluster deployment

from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute
from azureml.core.model import Model

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
import sklearn


environment = Environment.get(workspace=ws, name="supplychainray310", version=23)
aks_target = AksCompute(ws,"sa-aks-f8v2")
inference_config = InferenceConfig(entry_script='score_kube_update.py', source_directory= "../source", environment=environment)
aks_service_name="supplychainrayp104"
service = AksWebservice(ws,aks_service_name)
service.delete()
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 3, scoring_timeout_ms=300000)
service = Model.deploy(ws, aks_service_name, [], inference_config, deployment_config, aks_target,overwrite=True)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.get_logs())


/tmp/ipykernel_3612233/3343642732.py:23: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, aks_service_name, [], inference_config, deployment_config, aks_target,overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-17 20:49:08+00:00 Creating Container Registry if not exists.
2023-05-17 20:49:08+00:00 Registering the environment.
2023-05-17 20:49:09+00:00 Use the existing image.
2023-05-17 20:49:11+00:00 Creating resources in AKS.
2023-05-17 20:49:11+00:00 Submitting deployment to compute.
2023-05-17 20:49:12+00:00 Checking the status of deployment supplychainrayp104.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
/bin/bash: /azureml-envs/azureml_9f599a7a28bc5c7382f9f35240507922/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_9f599a7a28bc5c7382f9f35240507922/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_9f599a7a28bc5c7382f9f35240507922/lib/libtinfo.s

In [34]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {}

config ={'Location':'Location_0'}
data = {"Inputs":config}
body = str.encode(json.dumps(data))


body = str.encode(json.dumps(data))

url = 'YOUR_ONLINE_ENDPOINT/api/v1/service/supplychainrayp104/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = '3tJLCQvMF3Gn6zMUYh2U98sEt4MyRgtx'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}


req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'"Optimization done! \\nResult: \\n\\nnull"'


Bad pipe message: %s [b'\xd3o\xb8\xbe\xef6\x15i\xd9\xfd\xe5#s\xab`\x96\xa7  \x11\x87Q\x0bv;[\n\xb5 \xe5=\x01\x87n\xa8<\xfe=G\xdf\x9aM\x87\x02\xff\xdcM\x01z\xbd\xc4\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01', b'\n']
Bad pipe message: %s [b'\x8b\xb2\xda\x05X$\xbdR\xd8?\x03u\xba\x1630\xfd$\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0']
Bad pipe message: %s [b"\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00